In [1]:
pip install streamlit pandas scikit-surprise


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 19.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.8 MB/s eta 0:00:00
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162657 sha256=56c3aa03a0996459ffb77b59b89a36b36ddb084a699836c98d0c60f749a963df
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import streamlit as st
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# Load the dataset
df = pd.read_csv('final_ratings.csv')

# Load the Surprise dataset
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(df[['User-ID', 'ISBN', 'Book-Rating']], reader)

# Train-test split
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Build the SVD model
model = SVD()
model.fit(trainset)

# Evaluate the model
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)

# Streamlit App
st.title('Book Recommendation System')

# User input for Book ISBN
book_isbn = st.text_input('Enter a book ISBN to get recommendations:', '0316666343')

# Get recommendations for the entered book ISBN
if st.button('Get Recommendations'):
    try:
        # Get inner id for the entered book ISBN
        book_inner_id = model.trainset.to_inner_iid(book_isbn)

        # Get top N recommendations
        recommendations = model.get_neighbors(book_inner_id, k=5)

        # Convert inner ids back to ISBNs
        recommended_books = [model.trainset.to_raw_iid(inner_id) for inner_id in recommendations]

        # Display recommendations
        st.subheader('Top 5 Recommendations:')
        for i, book in enumerate(recommended_books):
            st.write(f"{i+1}. {df[df['ISBN'] == book]['Book-Title'].values[0]} by {df[df['ISBN'] == book]['Book-Author'].values[0]}")

    except ValueError:
        st.error('Invalid Book ISBN. Please enter a valid ISBN.')

# Display RMSE of the model
st.sidebar.subheader('Model Evaluation')
st.sidebar.text(f'RMSE: {rmse:.4f}')

# Optionally, you can display other information from the dataset in the main content or sidebar.
# Example: Display unique book titles in the sidebar
st.sidebar.subheader('Unique Book Titles')
unique_titles = df['Book-Title'].unique()
st.sidebar.write(unique_titles[:10])  # Displaying the first 10 unique titles as an example


2024-01-16 18:52:13.438 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]


RMSE: 1.5189
